In [1]:
!pip install konlpy
import re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 3.8MB 58.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [6]:
reviews = pd.read_csv('drive/My Drive/datasets/review_final.csv')
reviews = reviews.loc[:,['댓글내용']]

In [7]:
reviews.sample(frac=1).reset_index(drop=True)

,댓글내용
0,분.쇄.육. 무엇이 들어갔는진 비.밀.대충 상단 추천 많이받은것만 보고 구매했더니만...
1,기대가컷나봐용 제입에는 그닥 ~맛 만족도 예상보다 맛 없어요
2,에어프라이어에 기름없이 돌렸는데\n기름이 많이 나왔네요 ㅎ
3,5살 아이가 한입 먹고 하는 말\n배달 아저씨가 가져다 주는 치즈스틱이 더 맛있어요...
4,너무많이짜고 닭비린내납니다\n짠거좋아하는남편이 짜서못먹겠다고하내요\n재구매안합니다맛...
...,...
87406,부대찌개 구입해서 먹는것 처음인데\n이정도 퀄리티라면 자주사먹을것 같아요\n신랑이 ...
87407,굿♡\n밥알이살아있어요~~\n계란 스크렘블 꼭 해서 같이섞어드세요\n진짜맛있구요 계...
87408,굿
87409,간편히먹을수있어좋아요 맛나요


In [8]:
!pip install chatspace

     |████████████████████████████████| 10.7MB 6.6MB/s 


In [9]:
# 띄어쓰기 보정 오픈소스
from chatspace import ChatSpace

spacer = ChatSpace()
print(reviews['댓글내용'][0])
spacer.space(reviews['댓글내용'][0].replace('\n',' '))

Loading JIT Compiled ChatSpace Model
포장지 열자마자 유부초밥냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던건지
아니면 배송하시는 분이 던져서 터진건지는 모르겠지만
별로네요 그냥 다 버려서 맛은 모르겠어요맛 만족도 괜찮아요


'포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만 별로네요 그냥 다 버려서 맛은 모르겠어요 맛 만족도 괜찮아요'

In [36]:
test_data = []
for text in reviews['댓글내용']:
  test_data.append(spacer.space(text).replace('\n',''))
test_data

['포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만  별로네요 그냥 다 버려서 맛은 모르겠어요 맛 만족도 괜찮아요',
 '세 번째 시켜 먹었는데 깜짝 놀랐네요. 두부 넣어서 먹는 게 맛있어서 이번에도 시켰는데  유부초밥에서 플라스틱이 나왔네요.. 만약에 손님 대접하고 씹어먹다가 걸렸으면 어쩔 뻔 했는지..  믿고 시켰는데 화나고, 당황스러워서 여기에 평 남겨요. 신선도 예상만큼 신선해요',
 '안에 들어있는 유부팩 하나가 터져서 국물이 냉장고 안에 줄줄 흘렀어요.. 안에서 터진거라 알 수 없었겠지만 냉장고 강제 청소를 했네요 ㅡ.ㅡ반품 신청 하기 번거로워 그냥 폐기처분 했습니다..',
 '삼각은 싸서 입구를 막는데 사각은 토핑용인듯요 삼각보다 불편해요... ㅠㅠ',
 '맛이 좋아요 맛 만족도 예상보다 맛있어요',
 '유부가 조금 더 크면 좋겠어요 맛있네요 저한테는 좀 유부가 작네요 치즈랑 같이 먹으니 더 맛있어요',
 '유부초밥 비법은 밥에 양!! 전비법을 모름ㅋ 큰 사각 유부를 찾고 있었는데 시중에 파는 유부초밥은 삼각!! 이건 그냥 사각으로 자른 삼각 유부와 비슷한 사이즈네요~ 유부초밥도 회사마다 은근 맛차이 있던데, 어디 회사 유부가 진짜 맛이 없었는데 어딘지 생각이 안나는데 이건 맛있네요. 유부 초밥 만들 때마다 고민하는 게 밥양인데 이번에도 역시 밥이 남았어용 ㅋ 실생활 밥양 표기가 됐으면 좋겠어요 ㅋ 한그릇 반을 두그릇을 준시하시오. 이런식으로 ㅋ',
 '맛있지만 사각유부라는 거 꼭 생각하고 구매하세요 유부의 맛은 문제가 안됩니다. 맛음 맛있고 좋은데, 저는 사각이라는 모양이 별로인 것 같아요. 만들기가 너무 불편하고,  그렇다고 먹을 때 더 편하다? 그것도 모르겠어요. 유부 맛은 맛있습니다. 모양이 사각이라는 거 꼭 생각하시고 구매하세용 저는 다시 구매할 의향은 없구요, 세모 유부살래요....',
 '식초가 너무 시큼해서 호불호가 

In [227]:
test_data = pd.DataFrame(test_data, columns=['review'])
test_data

,review
0,포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 ...
1,세 번째 시켜 먹었는데 깜짝 놀랐네요. 두부 넣어서 먹는 게 맛있어서 이번에도 시켰...
2,안에 들어있는 유부팩 하나가 터져서 국물이 냉장고 안에 줄줄 흘렀어요.. 안에서 터...
3,삼각은 싸서 입구를 막는데 사각은 토핑용인듯요 삼각보다 불편해요... ㅠㅠ
4,맛이 좋아요 맛 만족도 예상보다 맛있어요
...,...
87406,편리성 사용하기 편리해요 신선도 예상만큼 신선해요 맛 만족도 예상보다 맛있어요
87407,편리성 그저 그래요 신선도 예상보다 신선해요 맛 만족도 괜찮아요
87408,편리성 그저 그래요 신선도 예상만큼 신선해요 맛 만족도 괜찮아요
87409,신선도 예상보다 신선해요 맛 만족도 예상보다 맛있어요


In [123]:
keyword_list = {'맛': ['맛','불맛', '감칠맛', '쓴맛', '향', '단맛', '신맛', '냄새', '끝맛', '짠맛', '꼬순내', '햄맛', '양이',
                      '햄향', '닭맛', '불향', '풋내', '조미료', '끝향', '쏘스맛', '닭내', '파맛', '존맛', '핵존맛', '텁텁한맛'],
                '가격':['가격', '금액', '값', '단가', '할인', '가성', '득템', '창렬', '혜자', '정가', '사기'],
                '양':['용량', '양은', '양도', '양으로', '개수', '인분', '분량', '물량', '사이즈', '수량','양이', '남았', '남아','많'],
                '품질':['품질', '상태', '아이스', '얼음', '비닐', '아이스팩', '뾱뾱', '뽁뽁이팩','드라이아이스', '아이스박스', '보냉', '상자',
                      '종이팩', '종이', '상온', '실온','상한','상했','썩','재료',
                        '소재', '박스', '뽁뽁이', '비닐봉투', '밀봉', '보냉팩', '배송', '발송', '택배', '배달', '검사', '퀄리티', '퀄러티','환경',
                       '검증', '신뢰', '신선',  '유통기한', '유효기한', '싱싱', '상한', '부패', '냄새'],
                '간편':['간편', '간단', '초보', '방법', '쉬움', '어려움', '단시간', '출퇴근시간', '간편식', '전문식','편리', '손쉽', '혼밥', '꽝손', 
                      '불편', '똥손', '금손', '편의'],
                '외관':['외관', '육안', '디자인', '실물', '겉보기', '보고', '보임', '볼때', '볼 때', '보고', '맛보다', '봣', '눈으로', '사진찍기 좋은',
                      '먹음직한', '말쌀나는', '푸디용', '색감', '색깔', '색이', '모양새', '이쁘', '예쁘','이쁜', '예쁜', '이뻐요', '예뻐요'],
                '웰빙':['웰빙', 'msg', '인스턴트', '건강식', '특수식품', '즉석식품', '건강', '다이어트', '다이어터', '헬스', '무첨가', '식단', '단백질',
                      '영양', '효과', '예방', '체중', '영양소', '탄수화물', '칼로리', '성분', '소화효소', '체중감량', '면역', '근육', '야맹증',
                      '질환', '보조', '미용', '운동', '에너지', '호르몬', '칼슘', '성인병', '갱년기', '성장', '비료', '노화', 
                      '치매', '효능', '염분', '두뇌', '불균형', '뇌', '알츠하이머', '코로나', '영양분', '소화']}

In [230]:
for keyword in keyword_list:
  test_data[keyword] = -1
test_data

,review,맛,가격,양,품질,간편,외관,웰빙
0,포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 ...,-1,-1,-1,-1,-1,-1,-1
1,세 번째 시켜 먹었는데 깜짝 놀랐네요. 두부 넣어서 먹는 게 맛있어서 이번에도 시켰...,-1,-1,-1,-1,-1,-1,-1
2,안에 들어있는 유부팩 하나가 터져서 국물이 냉장고 안에 줄줄 흘렀어요.. 안에서 터...,-1,-1,-1,-1,-1,-1,-1
3,삼각은 싸서 입구를 막는데 사각은 토핑용인듯요 삼각보다 불편해요... ㅠㅠ,-1,-1,-1,-1,-1,-1,-1
4,맛이 좋아요 맛 만족도 예상보다 맛있어요,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
87406,편리성 사용하기 편리해요 신선도 예상만큼 신선해요 맛 만족도 예상보다 맛있어요,-1,-1,-1,-1,-1,-1,-1
87407,편리성 그저 그래요 신선도 예상보다 신선해요 맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
87408,편리성 그저 그래요 신선도 예상만큼 신선해요 맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
87409,신선도 예상보다 신선해요 맛 만족도 예상보다 맛있어요,-1,-1,-1,-1,-1,-1,-1


In [193]:
# tokenizing
okt = Okt()
def tokenize(doc):
    doc = spacer.space(doc)
    # 한글 자음, 모음 제거
    doc = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=doc)
    # 특수기호 제거
    doc = re.sub(pattern='[^\w\s]', repl='', string=doc)
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    doc = okt.pos(doc, norm=True, stem=True)
    # 조사 제외
    token = []
    for i in doc:
        if i[1] != 'Josa':
          token.append(i)

    return ['/'.join(t) for t in token]

In [194]:
def vectorize(sentence, encoder):
  l = []
  token = tokenize(sentence)
  l.append(token)
  result = encoder.texts_to_sequences(l)
  return pad_sequences(result, 300)

In [195]:
# 인코더 기존 학습된 모델 사용
import pickle
with open('drive/My Drive/datasets/encoder_v3.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [220]:
def delete_token(q, poomsa):
    for j in range(0, len(q)):
        if q[j][1] == poomsa:
            return 1
    return 0

def inject_keyword(text):
  # 결과값
  result = {'맛':'', '가격': '', '양':'', '품질':'', '간편':'', '외관':'', '웰빙':''}
  sentences = re.split('[,.;?!/~]+',text) # 문장 분리 by ",.;?!/~"

  # 맛 ~ 웰빙까지
  for key in keyword_list.keys():
    keys = keyword_list[key] # 키워드 + 연관 키워드 리스트
    highlight = []

    for k in keys: 
      for sentence in sentences:
        words = sentence.split(' ') # 각 문장 띄어쓰기 분리
        words = [s for s in words if s != ''] # ''삭제
        for word in words:
          if k in word:
            injected = ''
            index = words.index(word)
            # 키워드에 해당하는 단어의 인덱스를 기준으로 앞 뒤 몇 글자씩 잘라내서 추출
            # window 파라미터로 몇 글자를 잘라낼 것인지 설정
            # 키워드의 앞은 형용사일 때만 추출
            foreward = index - 1
            if foreward >= 0:
              if okt.pos(words[foreward])[-1][1] == 'Adjective':
                injected = words[foreward] + ' '

            injected += words[index] + ' '
            
            # 키워드의 뒤 아홉 단어 추출
            backward = words[index+1: index+10]
            # 아홉 단어 okt 적용 후
            token_list = {}
            for i in backward:
              token_list[i] = okt.pos(i)
            # 품사가 기호가 있으면 자르고 stop하고, 없으면 형용사찾아서 자름
            # 다 없으면 아홉 단어 그대로 들어감
            tmp_list = []
            for p, q in token_list.items():
              for poomsa in ["Punctuation", "Adjective"] :
                zero_one = delete_token(q, poomsa)
              tmp_list.append(p)
              if zero_one == 1:
                break;

            injected += " ".join(tmp_list)
            highlight.append(injected)
      
    result[key] = highlight
      
  return result

In [191]:
test_data['review'][0]

'포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 상품 포장상태가 안좋았던 건지 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만  별로네요 그냥 다 버려서 맛은 모르겠어요 맛 만족도 괜찮아요'

In [258]:
samples.loc[0,0]

''

In [264]:
# 테스트용 3000개 데이터 사용
samples = test_data[:3000]
for col in range(7):
  samples[col] = ''
# 사전 학습된 LSTM 모델 사용
model = load_model('drive/My Drive/model/LSTM_binary_v2.h5')
for ss, sample in enumerate(samples['review']):
  inject_result = inject_keyword(sample)
  for kk, key in enumerate(keyword_list.keys()):
    inject_list = inject_result[key]
    
    # 해당 키워드가 없으면 점수는 -1
    if not inject_list:
      samples.loc[ss, key] = -1
    else:
      score = 0
      for one_inject in inject_list:
        score += model.predict(vectorize(one_inject, encoder))[0][0]
      samples.loc[ss, key] = ( score / len(inject_list) ) * 100
      samples.at[ss, kk] = ', '.join(inject_list)
samples

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,review,맛,가격,양,품질,간편,외관,웰빙,0,1,2,3,4,5,6
0,포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 ...,83.057990,-1.000000,-1.0,30.732832,-1.000000,-1.0,-1.0,"맛은 모르겠어요 맛 만족도 괜찮아요, 맛 만족도 괜찮아요, 냄새가 나서 자세히",,,"포장상태가 안좋았던, 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만 별로네요...",,,
1,세 번째 시켜 먹었는데 깜짝 놀랐네요. 두부 넣어서 먹는 게 맛있어서 이번에도 시켰...,99.381000,-1.000000,-1.0,98.533159,-1.000000,-1.0,-1.0,맛있어서 이번에도 시켰는데 유부초밥에서 플라스틱이 나왔네요,,,"신선도 예상만큼 신선해요, 신선해요",,,
2,안에 들어있는 유부팩 하나가 터져서 국물이 냉장고 안에 줄줄 흘렀어요.. 안에서 터...,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.0,,,,,,,
3,삼각은 싸서 입구를 막는데 사각은 토핑용인듯요 삼각보다 불편해요... ㅠㅠ,-1.000000,-1.000000,-1.0,-1.000000,63.041371,-1.0,-1.0,,,,,불편해요,,
4,맛이 좋아요 맛 만족도 예상보다 맛있어요,99.628375,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.0,"맛이 좋아요, 좋아요 맛 만족도 예상보다 맛있어요, 맛있어요",,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,아이소 풍용으로 샀어요 베이컨 말이 밥을 넘 좋아해서요 안 찢어지구 잘 말아지네요 ...,99.735227,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.0,"맛 만족도 예상보다 맛있어요, 맛있어요",,,,,,
2996,1+1이라 마트보다 싼 가격에 잘 구매해서 맛나게 요리해 먹었어요 맛 만족도 예상보...,99.758943,99.860460,-1.0,-1.000000,-1.000000,-1.0,-1.0,"맛나게 요리해 먹었어요 맛 만족도 예상보다 맛있어요, 맛 만족도 예상보다 맛있어요,...",가격에 잘 구매해서 맛나게 요리해 먹었어요 맛 만족도 예상보다 맛있어요,,,,,
2997,베이컨 말이용으로 샀는데요~! 맛있고 부드럽고 좋았습니다. 맛 만족도 예상보다 맛있어요,99.794783,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.0,"맛있고 부드럽고, 맛 만족도 예상보다 맛있어요, 맛있어요",,,,,,
2998,브런치에 먹으려 샀는데 맛있네 묘. 조금 짰지만 계란 후라이랑 먹으면 간이 맞아요,99.269074,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.0,맛있네 묘,,,,,,


In [265]:
samples[samples['맛'] != -1]

,review,맛,가격,양,품질,간편,외관,웰빙,0,1,2,3,4,5,6
0,포장지 열자마자 유부초밥 냄새가 나서 자세히 봤더니 사진 처럼 국물이 고여있었어요 ...,83.057990,-1.00000,-1.000000,30.732832,-1.0,-1.0,-1.0,"맛은 모르겠어요 맛 만족도 괜찮아요, 맛 만족도 괜찮아요, 냄새가 나서 자세히",,,"포장상태가 안좋았던, 아니면 배송하시는 분이 던져서 터진 건지는 모르겠지만 별로네요...",,,
1,세 번째 시켜 먹었는데 깜짝 놀랐네요. 두부 넣어서 먹는 게 맛있어서 이번에도 시켰...,99.381000,-1.00000,-1.000000,98.533159,-1.0,-1.0,-1.0,맛있어서 이번에도 시켰는데 유부초밥에서 플라스틱이 나왔네요,,,"신선도 예상만큼 신선해요, 신선해요",,,
4,맛이 좋아요 맛 만족도 예상보다 맛있어요,99.628375,-1.00000,-1.000000,-1.000000,-1.0,-1.0,-1.0,"맛이 좋아요, 좋아요 맛 만족도 예상보다 맛있어요, 맛있어요",,,,,,
5,유부가 조금 더 크면 좋겠어요 맛있네요 저한테는 좀 유부가 작네요 치즈랑 같이 먹으...,99.853900,-1.00000,-1.000000,-1.000000,-1.0,-1.0,-1.0,"좋겠어요 맛있네요 저한테는 좀 유부가 작네요, 맛있어요",,,,,,
6,유부초밥 비법은 밥에 양!! 전비법을 모름ㅋ 큰 사각 유부를 찾고 있었는데 시중에 ...,76.262206,-1.00000,96.899414,-1.000000,-1.0,-1.0,-1.0,"맛차이 있던데, 맛이 없었는데, 맛있네요",,"비슷한 사이즈네요 , 남았어용 ㅋ 실생활 밥양 표기가 됐으면 좋겠어요",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,베이컨은 베이컨 베이컨 맛이야 거기서 거기라 이 베이컨도 그냥 평범한 한국 베이컨 ...,96.534733,-1.00000,98.379445,-1.000000,-1.0,-1.0,-1.0,"맛이야 거기서 거기라 이 베이컨도 그냥 평범한, 맛이에요 생각보다 양이 많아서, 양...",,"양이 많아서, 많아서 좋았어요",,,,
2995,아이소 풍용으로 샀어요 베이컨 말이 밥을 넘 좋아해서요 안 찢어지구 잘 말아지네요 ...,99.735227,-1.00000,-1.000000,-1.000000,-1.0,-1.0,-1.0,"맛 만족도 예상보다 맛있어요, 맛있어요",,,,,,
2996,1+1이라 마트보다 싼 가격에 잘 구매해서 맛나게 요리해 먹었어요 맛 만족도 예상보...,99.758943,99.86046,-1.000000,-1.000000,-1.0,-1.0,-1.0,"맛나게 요리해 먹었어요 맛 만족도 예상보다 맛있어요, 맛 만족도 예상보다 맛있어요,...",가격에 잘 구매해서 맛나게 요리해 먹었어요 맛 만족도 예상보다 맛있어요,,,,,
2997,베이컨 말이용으로 샀는데요~! 맛있고 부드럽고 좋았습니다. 맛 만족도 예상보다 맛있어요,99.794783,-1.00000,-1.000000,-1.000000,-1.0,-1.0,-1.0,"맛있고 부드럽고, 맛 만족도 예상보다 맛있어요, 맛있어요",,,,,,


In [266]:
samples[samples['가격']!= -1]

,review,맛,가격,양,품질,간편,외관,웰빙,0,1,2,3,4,5,6
9,"토핑한 유부초밥이 안쓰러지고 이쁘게 도시락 싸려면 사각 유부초밥이 좋을 것 같고, ...",37.392136,99.049592,-1.000000,-1.000000,-1.000000,99.904054,-1.0,"맛도 없고, 맛을 살렸어요, 없습니다 맛 만족도 예상보다 맛 없어요, 없습니다 맛 ...",가격도 되게 저렴해서,,,,안쓰러지고 이쁘게 도시락 싸려면 사각 유부초밥이 좋을,
34,괜찮아요 맛있어요 요즘 유부초밥에 꽂혀서 이틀에 한번씩 해먹네요 가격은 싸서 좋...,98.208097,98.984104,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,괜찮아요 맛있어요 요즘 유부초밥에 꽂혀서 이틀에 한번씩 해먹네요 가격은 싸서 좋은데...,가격은 싸서 좋은데,,,,,
45,가성비 좋아요 가성비 좋아요,-1.000000,99.806494,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,,"가성비 좋아요, 가성비 좋아요",,,,,
78,"아 이용, 도시락용 추천 맛있어요! 구매일 20. 08. 15 제조일 20. 08....",98.946887,83.369821,95.616262,97.435302,99.947876,-1.000000,-1.0,"맛있어요 , 편하고 맛도 있고, ●맛새콤달콤한 유부초밥의 정석인 맛이예요, 맛이예요...",가격 3,"5인분이라고 되어있는데 우리 가족은 많이 먹는 편이라 성인2, 많이 가거든요, 많이...","상온으로 먹는 게 맛있는, 유통기한 20",간단하기 끼니 떼우는 용이면 그냥 큰 유부초밥 사세요 ㅎㅎㅎ 만들기,,
80,간 편하게 한입 쏙 ! 사각 유부초밥~~ 결론부터 말하자면 세모 유부초밥보다 사각 ...,99.824643,98.135400,99.223483,99.367848,99.852377,-1.000000,-1.0,맛있었어요,가격도 저렴하구,"5인분이라 개봉하면 봉지가 두 개씩 나 뉘어져 있구요, 많이 짜지 않고 간간하니 너...","저렴하구 배송도 로켓 배송되니 이만큼 편한, 배송되니 이만큼 편한",간단히 따라 할 수 있는,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2982,베이컨은 항상 요기꺼만! 가격대비 참이나 양이 굿입니다! 파스타나 볶음밥 여기 저기...,99.822348,99.837792,99.822348,-1.000000,-1.000000,-1.000000,-1.0,양이 굿입니다,가격대비 참이나 양이 굿입니다,양이 굿입니다,,,,
2983,적당한 양에 좋은 가격 좋았어요. 파스타 하려고 구매했어요. 저는 가격도 적당하고 ...,99.914718,99.515259,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,적당하고 맛있게 잘 먹었네요 ^^,"좋은 가격 좋았어요, 가격도 적당하고",,,,,
2992,배송굿이네요 가격 제한 없이 무료 로켓 배송 계속 가즈아~~ 베이컨 가격 내려주세요...,-1.000000,99.419443,-1.000000,95.808902,-1.000000,-1.000000,-1.0,,"가격 제한 없이 무료 로켓 배송 계속 가즈아, 가격 내려주세요 우리 딸램 넘 좋아하...",,"배송굿이네요 가격 제한 없이 무료 로켓 배송 계속 가즈아, 배송 계속 가즈아",,,
2996,1+1이라 마트보다 싼 가격에 잘 구매해서 맛나게 요리해 먹었어요 맛 만족도 예상보...,99.758943,99.860460,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,"맛나게 요리해 먹었어요 맛 만족도 예상보다 맛있어요, 맛 만족도 예상보다 맛있어요,...",가격에 잘 구매해서 맛나게 요리해 먹었어요 맛 만족도 예상보다 맛있어요,,,,,


비고 
 * 아직까지 정확도에 한계가 있다. 키워드 추출 로직도 보완 필요.
 * 키워드 연관 단어 사전 역시 보완 필요
 * POC 나 프로토타입 때 보여주기 용으로는 나쁘지 않은듯.
 * 계산 시간 비용이 많이 든다. 3000개 학습 시 10분 정도, 9만 개 학습 시 5시간